# CS 4700 - Homework 3: Adversarial Search
<h3 style="color: red;">
    Due: Monday, March 4, 1:24pm on Gradescope
    <br><br>
    Late submission: Wednesday, March 6, 1:24pm on Gradescope (50% penalty)
</h3>

Which other students did you interact with on this assignment? Provide their NetID(s) below consistent with the [homework policy](http://www.cs.cornell.edu/courses/cs4700/2019sp/).

jd952, xl738

### Written Question

<h3 style="text-align: right; margin-top: -1em;">25 points</h3>

---

1. Consider the following game tree.  Each game state has two actions that can be taken: Left and Right.  The numbers at the terminal nodes are the values given by the evaluation function.
  1. Which move should be made at the top level (Left or Right)?  What is its value?
  1. Which nodes get pruned if you use alpha-beta pruning when you explore successors of states going left-to-right? 
  1. Give the answer to (B) when successors are explored right-to-left.
  
![](game_tree.png)

1A:<br/>
**Right** move should be made at the top level. The value is **5**. 

1B: <br/>
**[S, K, AA, O]**

1C:<br/>
**[D]**

2. Consider using minimax search with alpha–beta pruning on a game whose branching factor is $b$ and where you search to depth $m$. If successor nodes are searched in the worst-case order no pruning happens and the algorithm takes $O(b^m)$ time.  (Make sure you understand why that is true.) It has been shown that if successors are ordered in the best-case optimal fashion the algorithm’s time complexity will be $O(b^{\frac{m}{2}})$. This means the best-case scenario will take less time to search to the same depth $m$. Let’s say the worst-case ordering of successors runs for time $T$.  How much deeper can you search in the best-case scenario if it is also allowed to run for time $T$?  To make this as simple as possible, assume the constants that are “hidden” in both big-O time complexity expressions are the same.

The answer is **2m**.<br /><br />
In the worst case, searching depth *$m$* will result in *$O(b^m)$* time. We denote the searching depth by $d$ where we also take $T$ (i.e., $O(b^m)$) time in the best case. We then get $O(b^{\frac{d}{2}})=T=O(b^m)$. Hence, $d=2m$.  

### Programming Introduction
---

This part of the assignment will provide you with programming exercises to help cement the key ideas about adversarial search. For this purpose, we have chosen a simple 2-player strategy game called __Neutron__ as our domain of interest. It is played on a 5 x 5 board. Each player has 5 soldier pieces and a respective home row. The soldiers for each player start by occupying their home row; for player 1, this is row 1, and for player 2, this is row 5. In addition, there is a neutral piece called the Neutron located in the center of the board that both players control.

For our purposes, we have chosen to make each piece distinct from one another. The soldier pieces for player 1 are marked by the first 5 letters of the alphabet. The soldier pieces for player 2 are marked by the first 5 positive integers. The Neutron piece is marked by a \*. The initial position of the board (as well row and column numbers) can be illustrated as:

```
          1   2   3   4   5

        ---------------------
1       | A | B | C | D | E |
        ---------------------
2       |   |   |   |   |   |
        ---------------------
3       |   |   | * |   |   |
        ---------------------
4       |   |   |   |   |   |
        ---------------------
5       | 1 | 2 | 3 | 4 | 5 |
        ---------------------
```

__RULES:__ The game begins with player 1 and then alternates between players. In the first turn, player 1 must select 1 of their own soldier pieces to move. Then, for every turn after this, a player must make 2 sequential moves. First, they must move the Neutron piece. Second, they must select 1 of their own soldier pieces to move. The movement of a piece (soldier or Neutron) can be in either of the 4 directions: up, left, down, right. A piece can move in a direction as long as the path is clear. A piece cannot take another piece, jump over a piece, or otherwise change directions. They must move as many spaces as possible in the direction chosen until they collide with another piece or a wall.

For example, let us use the initial state illustrated above. Since this is the beginning of the game, player 1 has 5 available choices. Note, coordinates are in (row,column) format.

1. Move A from (1,1) down to (4,1)
1. Move B from (1,2) down to (4,2)
1. Move C from (1,3) down to (2,3)
1. Move D from (1,4) down to (4,4)
1. Move E from (1,5) down to (4,5)

Let's say player 1 decides to use move 4 from the list above. The resulting state would be:

```
          1   2   3   4   5

        ---------------------
1       | A | B | C |   | E |
        ---------------------
2       |   |   |   |   |   |
        ---------------------
3       |   |   | * |   |   |
        ---------------------
4       |   |   |   | D |   |
        ---------------------
5       | 1 | 2 | 3 | 4 | 5 |
        ---------------------
```

It is now player 2's turn. Since this is no longer the initial state of the game, the player must make 2 sequential moves (Neutron followed by a soldier). Therefore, player 2 has 15 possible actions to choose from:

1. Move * from (3,3) up to (2,3), then move 1 from (5,1) up to (2,1)
1. Move * from (3,3) up to (2,3), then move 2 from (5,2) up to (2,2)
1. Move * from (3,3) up to (2,3), then move 3 from (5,3) up to (3,3)
1. Move * from (3,3) up to (2,3), then move 5 from (5,5) up to (2,5)
1. Move * from (3,3) left to (3,1), then move 1 from (5,1) up to (4,1)
1. Move * from (3,3) left to (3,1), then move 2 from (5,2) up to (2,2)
1. Move * from (3,3) left to (3,1), then move 3 from (5,3) up to (2,3)
1. Move * from (3,3) left to (3,1), then move 5 from (5,5) up to (2,5)
1. Move * from (3,3) down to (4,3), then move 1 from (5,1) up to (2,1)
1. Move * from (3,3) down to (4,3), then move 2 from (5,2) up to (2,2)
1. Move * from (3,3) down to (4,3), then move 5 from (5,5) up to (2,5)
1. Move * from (3,3) right to (3,5), then move 1 from (5,1) up to (2,1)
1. Move * from (3,3) right to (3,5), then move 2 from (5,2) up to (2,2)
1. Move * from (3,3) right to (3,5), then move 3 from (5,3) up to (2,3)
1. Move * from (3,3) right to (3,5), then move 5 from (5,5) up to (4,5)

This back-and-forth will continue until the game ends. So, how does the game of Neutron end?

__GOALS:__ The main goal of the game is for a player to move the Neutron piece to their home row. This can happen in a couple of ways. Either the player is able to move the Neutron on their own turn to their home row, or can otherwise force the opponent into moving the Neutron to the player's home row. Alternatively, it is also possible to win the game without securing the Neutron by blocking your opponent from moving, i.e. when your opponent is deadlocked, you also win.

Run the code cell below which contains the logic for __Neutron__. Feel free to look through the code for a better understanding of the game implementation.

In [1]:
# import statements
import math
import random
import time
from collections import namedtuple

# additional definitions
GameState = namedtuple('GameState', 'board, player, is_initial, utility_value')
PLAYER_1  = ['A', 'B', 'C', 'D', 'E']
PLAYER_2  = ['1', '2', '3', '4', '5']
NEUTRON   = '*'
BLANK     = ' '

"""
The Neutron game. Implemented as a class so that multiple instances can be created.

A state for this game is a 4-tuple, (board, player, is_initial, utility_value).
- board, a dictionary with (i,j) coordinates as the key, and the piece as
  the value. Valid coordinate values for i and j are 1 to 5, inclusive. The piece
  should refer to one of the variables defined above.
- player, refers to the player that needs to move in this state. Should reference
  one of the variables defined above.
- is_initial, a Boolean value that determines whether or not this state is the
  initial state of the game.
- utility_value, an integer value that represents the utility value for the
  state.
"""
class NeutronGame:
    """
    Constructor for the class. Initializes the initial state of the game, the move
    ordering that should be used, and the number of states generated during a game.
    The game starts with all of player 1's pieces at row 1, all of player 2's pieces
    at row 5, and the neutron in the center. The fixed move ordering is up, down,
    left, then right.
    
    Optional parameters:
    - custom_state, a user-provided state to use as the initial state of the game
    - move_order, a custom move order to be used in actions()
    """
    def __init__(self, custom_state = None, move_order = None):
        if custom_state:
            self.initial_state = custom_state
        else:
            board = {(1, j + 1) : PLAYER_1[j] for j in range(5)}
            board.update({(i,j) : BLANK for j in range(1,6) for i in range(2,5)})
            board.update({(5, j + 1) : PLAYER_2[j] for j in range(5)})
            board[(3,3)] = NEUTRON
            self.initial_state = GameState(board = board, player = PLAYER_1,
                is_initial = True, utility_value = 0)
        
        if move_order:
            self.move_order = move_order
        else:
            self.move_order = [(-1,0), (1,0), (0,-1), (0,1)]
        
        self.num_states_gen = 0

    """
    Given a well-defined state as input, this function will return all the
    possible actions that can be taken as a list. An action can consist of
    1 or 2 moves, and so is treated as a list. A move is represented as a
    tuple (piece, description), where piece refers to one of the global
    variables and description is a string describing the movement of the piece.
    """
    def actions(self, state):
        acts = []

        # must move the neutron piece first if this is not the initial state
        neutron_moves = []
        if not state.is_initial:
            neutron_moves = [self.try_move(state.board, NEUTRON, i, j) \
                             for (i,j) in self.move_order]
            neutron_moves = filter(lambda m : m != None, neutron_moves)
            neutron_moves = map(lambda m : self.simplify_move(state.board, m),
                neutron_moves)

        # if the neutron was moved, must temporarily transition the state first
        # then move the soldier pieces
        for nm in neutron_moves:
            temp_board = self.result(state, [nm]).board
            for piece in state.player:
                moves = [self.try_move(temp_board, piece, i, j) \
                         for (i,j) in self.move_order]
                moves = filter(lambda m : m != None, moves)
                moves = map(lambda m : self.simplify_move(temp_board, m), moves)
                for m in moves:
                    acts.append([nm, m])

        # otherwise, just move the soldier pieces for the player
        if not neutron_moves:
            for piece in state.player:
                moves = [self.try_move(state.board, piece, i, j) \
                         for (i,j) in self.move_order]
                moves = filter(lambda m : m != None, moves)
                moves = map(lambda m : self.simplify_move(state.board, m),
                    moves)
                for m in moves:
                    acts.append([m])

        return acts

    """
    Try moving the piece on the board by shifting the piece's coordinates
    by the amount (delta_i,delta_j). Return the resulting move.
    """
    def try_move(self, board, piece, delta_i, delta_j):
        move   = None
        (i,j)  = self.find_piece(board, piece)
        temp_i = i + delta_i
        temp_j = j + delta_j
        while temp_i >= 1 and temp_i <= 5 and temp_j >= 1 and temp_j <= 5 \
        and board[(temp_i,temp_j)] == BLANK:
            temp_i += delta_i
            temp_j += delta_j
        temp_i -= delta_i
        temp_j -= delta_j
        if board[(temp_i,temp_j)] == BLANK:
            move = (piece, (temp_i,temp_j))
        return move

    """
    Find the piece on the board and return its coordinates.
    """
    def find_piece(self, board, piece):
        i, j = None, None
        for (m,n) in iter(board):
            if board[(m,n)] == piece:
                i, j = m, n
                break
        return (i,j)

    """
    Given move as the tuple (piece, description), convert the description into
    delta values to try moving the piece on the given board. Return the
    resulting move.
    """
    def decipher_move(self, board, move):
        (piece,desc) = move
        (i,j)        = self.find_piece(board, piece)
        vert         = {'UP' : -1, 'DOWN' : 1}
        horiz        = {'LEFT' : -1, 'RIGHT' : 1}
        return self.try_move(board, piece, vert.get(desc,0), horiz.get(desc,0))
        
    """
    Given a move in the format (piece, (i,j)), convert the movement into a simple
    description that is user-friendly.
    """
    def simplify_move(self, board, move):
        (piece, (i,j)) = move
        (m,n)          = self.find_piece(board, piece)
        if i < m:
            desc = 'UP'
        elif i > m:
            desc = 'DOWN'
        elif j < n:
            desc = 'LEFT'
        else:
            desc = 'RIGHT'
        return (piece,desc)

    """
    Given a state and an action, return the new state that our game should
    transition to. Assumes state and action are valid.
    """
    def result(self, state, action):
        board                = state.board.copy()
        player               = state.player
        self.num_states_gen += 1
        if not action:
            return GameState(board = board, is_initial = False, player = player,
                utility_value = 1 if state.player == PLAYER_2 else -1)
        for a in action:
            (piece, (i,j)) = self.decipher_move(board, a)
            (m,n)          = self.find_piece(board, piece)
            board[(m,n)]   = BLANK
            board[(i,j)]   = piece
        return GameState(board = board, is_initial = False,
            player = PLAYER_2 if player == PLAYER_1 else PLAYER_1,
            utility_value = self.compute_utility(board))

    """
    Return the utility of the given state from the perspective of
    the given player.
    """
    def utility(self, state, player):
        u = state.utility_value
        if u == 0 and not self.actions(state):
            return -1 if state.player == player else 1
        return u if player == PLAYER_1 else -u

    """
    Compute the utility of the board from the perspective of player 1.
    +1 if the neutron is in row 1, -1 if the neutron is in row 5,
    0 otherwise.
    """
    def compute_utility(self, board):
        (i,j) = self.find_piece(board, NEUTRON)
        u     = 0
        if i == 1:
            u = 1
        elif i == 5:
            u = -1
        return u

    """
    A state is terminal if the utility value is non-zero or if
    there are no available actions to take in that state.
    """
    def terminal_test(self, state):
        return state.utility_value != 0 or not self.actions(state)

    """
    Print the state to console.
    """
    def display(self, state):
        board_size = 5
        s          = '\n'
        for _ in range(board_size * 4 + 1):
            s += '-'
        print(s)

        for i in range(1, board_size + 1):
            s = ''
            for j in range(1, board_size + 1):
                s += '| ' + state.board[(i,j)] + ' '
            s += '|\n'
            for _ in range(board_size * 4 + 1):
                s += '-'
            print(s)

        print('Player to move:',
              'Player 1' if state.player == PLAYER_1 else 'Player 2')

    """
    Let two agents play the game from the initial state. display_flag
    toggles whether or not to show every state during gameplay.
    experiment_flag will suppress all output.
    
    These are the 4 possible outputs:
    (1,1) means player 2 wins, caused by player 1
    (1,2) means player 1 wins, caused by player 2
    (-1,1) means player 1 wins, caused by player 1
    (-1,2) means player 2 wins, caused by player 2
    """
    def play_game(self, first_player, second_player, display_flag, experiment_flag = False):
        state = self.initial_state
        if display_flag:
            self.display(state)
        while True:
            for player in [first_player, second_player]:
                action = player.play_turn(self, state)
                state  = self.result(state, action)
                if display_flag:
                    self.display(state)
                if self.terminal_test(state):
                    if not experiment_flag:
                        print('\nGame is over!')
                        self.display(state)
                    return (self.utility(state, state.player),
                            1 if player == first_player else 2)

Let's introduce 2 types of agents that will be able to play our game. The first, called `QueryAgent`, acts as a stand-in for the user. It will output the possible actions that the user can take and ask for input on which action to undergo. The second, called `RandomAgent`, simply chooses amongst the available actions at random. Note that an action in our implementation is a list of tuples (wherein a tuple represents a single move). Therefore, the input you provide the agent should also be a list of tuples.

For example, reusing our example from the introduction, the actions for player 1 at the initial state would be displayed as:

`[[('A', 'DOWN')], [('B', 'DOWN')], [('C', 'DOWN')], [('D', 'DOWN')], [('E', 'DOWN')]]`

Suppose player 1 decides to move its soldier piece B down. Then, as input, it would supply: `[('B', 'DOWN')]`

In [2]:
"""
An agent that relies on user input.
"""
class QueryAgent:
    def play_turn(self, game, state):
        action = None
        print('\nAvailable actions:', game.actions(state))
        if game.actions(state):
            action_str = input('\nYour action: ')
            try:
                action = eval(action_str)
            except NameError:
                action = action_str
        else:
            print('No legal actions remain!')
        return action

"""
An agent that will randomly choose amongst the available actions.
"""
class RandomAgent:
    def play_turn(self, game, state):
        return random.choice(game.actions(state)) if game.actions(state) else None

We now have a game and agents that can play it. To ensure you have a proper understanding of __Neutron__, let's have you play against a random agent. Run the code cell below to try your best to win!

In [3]:
game = NeutronGame()
p1   = QueryAgent()
p2   = RandomAgent()
game.play_game(p1, p2, True)


---------------------
| A | B | C | D | E |
---------------------
|   |   |   |   |   |
---------------------
|   |   | * |   |   |
---------------------
|   |   |   |   |   |
---------------------
| 1 | 2 | 3 | 4 | 5 |
---------------------
Player to move: Player 1

Available actions: [[('A', 'DOWN')], [('B', 'DOWN')], [('C', 'DOWN')], [('D', 'DOWN')], [('E', 'DOWN')]]

Your action: [('B', 'DOWN')]

---------------------
| A |   | C | D | E |
---------------------
|   |   |   |   |   |
---------------------
|   |   | * |   |   |
---------------------
|   | B |   |   |   |
---------------------
| 1 | 2 | 3 | 4 | 5 |
---------------------
Player to move: Player 2

---------------------
| A |   | C | D | E |
---------------------
|   |   | 3 |   |   |
---------------------
|   |   |   |   | * |
---------------------
|   | B |   |   |   |
---------------------
| 1 | 2 |   | 4 | 5 |
---------------------
Player to move: Player 1

Available actions: [[('*', 'UP'), ('A', 'DOWN')], [('*', 'U

(1, 2)

Finally, just for fun, let's have 2 random agents go at it. Toggle the display flag (third parameter for `play_game`) to see the full sequence of states for the game.

In [4]:
game = NeutronGame()
p1   = RandomAgent()
p2   = RandomAgent()
game.play_game(p1, p2, False) # can toggle display flag


Game is over!

---------------------
|   | B | C |   |   |
---------------------
|   |   | 1 |   |   |
---------------------
| 5 |   |   |   | 4 |
---------------------
| 2 |   |   |   | E |
---------------------
| A | 3 | * |   | D |
---------------------
Player to move: Player 1


(-1, 2)

### Part 1: Minimax and Maximin

<h3 style="text-align: right; margin-top: -1em;">20 points</h3>

---

As you learned in lecture, the __minimax__ and __maximin__ search algorithms provide a way for an agent to search an adversarial search space. Specifically, player 1 (traditionally called _MAX_) wants to maximize their score while player 2 (similarly referred to as _MIN_) wishes to minimize such a score. If we envision the search as a tree, then each depth corresponds to different responses (actions) that the current player can make to the opponent's previous action. As we descend this tree, the current player alternates until the search reaches a terminal state.

We can summarize this procedure as the following:

$$
\mbox{Minimax}(s) =
\begin{cases} 
      \mbox{Utility}(s) & \mbox{if Terminal}(s) \\
      argmax_{a\ \in \ \mbox{Actions}(s)} \mbox{Maximin(Result}(s, a)) & \mbox{if Player}(s) = \mbox{MAX} \\
      argmin_{a\ \in \ \mbox{Actions}(s)} \mbox{Minimax(Result}(s, a)) & \mbox{if Player}(s) = \mbox{MIN}
   \end{cases}
$$

Your first task in this assignment is to implement the pair of search algorithms. For easy reference, a modified pseudocode from lecture has been reproduced below. Note that, for this assigment, our algorithms must output two things: __a value and an action__. One handy feature about Python is that functions are allowed to return 2 outputs, so this works out. You can do this by using the syntax: `return val_1, val_2`. Lastly, remember that you __must__ also provide documentation for your solution.

```
minimax(game, state, cutoff, depth, eval_fn)
    if cutoff(state, depth) then
        return eval_fn(state, player 1), empty_action
    else
        val = -∞
        best_act = empty_action
        foreach act in actions(state)
            val' = maximin(game, result(state, act), cutoff, depth + 1, eval_fn)
            if val' > val then
                val = val'
                best_act = act
    return val, best_act
```

---

```
maximin(game, state, cutoff, depth, eval_fn)
    if cutoff(state, depth) then
        return eval_fn(state, player 2), empty_action
    else
        val = ∞
        best_act = empty_action
        foreach act in actions(state)
            val' = minimax(game, result(state, act), cutoff, depth + 1, eval_fn)
            if val' < val then
                val = val'
                best_act = act
    return val, best_act
```

In [5]:
"""
:Minimax search algorithm implentation
:type game: object (an object generated by NeutronGame class)
:type state: list (possible sates)
:type cutoff: function (optional, customized cutoff function, default by game.terminal_test)
:type depth: int (optional, recursion depth, default by 0)
:type eval_fn: function (optional, default by game.utility)
:rtype: tuple (returns a value and an action)
"""
def minimax(game, state, cutoff = None, depth = 0, eval_fn = None):
    # default cutoff is just a test if the state is terminal
    if cutoff == None:
        cutoff = lambda s, d : game.terminal_test(s)
    
    # default eval is just the utility value
    if eval_fn == None:
        eval_fn = game.utility
    
    # replace the line below with your code
    if cutoff(state, depth):
        return eval_fn(state, p1), []
    else:
        val = float('-inf')
        best_act = []
        for act in game.actions(state):
            val_curr = maximin(game, game.result(state, act), cutoff, depth+1, eval_fn)[0]
            if val_curr > val:
                val = val_curr
                best_act = act
    return val, best_act

"""
:Maximin search algorithm implentation
:type game: object (an object generated by NeutronGame class)
:type state: list (possible sates)
:type cutoff: function (optional, customized cutoff function, default by game.terminal_test)
:type depth: int (optional, recursion depth, default by 0)
:type eval_fn: function (optional, default by game.utility)
:rtype: tuple (returns a value and an action)
"""
def maximin(game, state, cutoff = None, depth = 0, eval_fn = None):
    # default cutoff is just a test if the state is terminal
    if cutoff == None:
        cutoff = lambda s, d : game.terminal_test(s)
    
    # default eval is just the utility value
    if eval_fn == None:
        eval_fn = game.utility
    
    # replace the line below with your code
    if cutoff(state, depth):
        return eval_fn(state, p2), []
    else:
        val = float('inf')
        best_act = []
        for act in game.actions(state):
            val_curr = minimax(game, game.result(state, act), cutoff, depth+1, eval_fn)[0]
        if val_curr < val:
            val = val_curr
            best_act = act
    return val, best_act

Once we have a working implementation of __minimax__ and __maximin__, we can create agents based around these algorithms. Both agents can take optional parameters but we will focus on these parameters later in the assignment.

In [6]:
"""
An agent that utilizes the minimax search algorithm. Can take two optional parameters.
cutoff, that can be used to determine when to stop the search
eval_fn, which evaluates a state's value from the perspective of a given player
"""
class MinimaxAgent:
    def __init__(self, cutoff = None, eval_fn = None):
        self.cutoff  = cutoff
        self.eval_fn = eval_fn

    def play_turn(self, game, state):
        _, best_act = minimax(game, state, cutoff = self.cutoff, eval_fn = self.eval_fn)
        return best_act

"""
An agent that utilizes the maximin search algorithm. Can take two optional parameters.
cutoff, that can be used to determine when to stop the search
eval_fn, which evaluates a state's value from the perspective of a given player
"""
class MaximinAgent:
    def __init__(self, cutoff = None, eval_fn = None):
        self.cutoff  = cutoff
        self.eval_fn = eval_fn

    def play_turn(self, game, state):
        _, best_act = maximin(game, state, cutoff = self.cutoff, eval_fn = self.eval_fn)
        return best_act

As a test, we are going to start from a particular state partway through a game. This particular state was written so that the minimax agent will be able to playout until the end. Run the code cell below to check if your implementation is able to win.

In [7]:
custom_board = {
    (1,1) : PLAYER_1[0], (1,2) : BLANK, (1,3) : PLAYER_1[1], (1,4) : PLAYER_1[2], (1,5) : PLAYER_1[3],
    (2,1) : PLAYER_2[0], (2,2) : NEUTRON, (2,3) : PLAYER_2[1], (2,4) : BLANK, (2,5) : BLANK,
    (3,1) : BLANK, (3,2) : PLAYER_1[4], (3,3) : BLANK, (3,4) : BLANK, (3,5) : BLANK,
    (4,1) : BLANK, (4,2) : BLANK, (4,3) : BLANK, (4,4) : BLANK, (4,5) : PLAYER_2[2],
    (5,1) : BLANK, (5,2) : PLAYER_2[3], (5,3) : BLANK, (5,4) : PLAYER_2[4], (5,5) : BLANK,
}
custom_state = GameState(board = custom_board, player = PLAYER_1, is_initial = False, utility_value = 0)
game         = NeutronGame(custom_state)
p1           = MinimaxAgent()
p2           = RandomAgent()
game.play_game(p1, p2, True)


---------------------
| A |   | B | C | D |
---------------------
| 1 | * | 2 |   |   |
---------------------
|   | E |   |   |   |
---------------------
|   |   |   |   | 3 |
---------------------
|   | 4 |   | 5 |   |
---------------------
Player to move: Player 1

---------------------
| A | * | B |   | D |
---------------------
| 1 |   | 2 |   |   |
---------------------
|   | E |   |   |   |
---------------------
|   |   |   | C | 3 |
---------------------
|   | 4 |   | 5 |   |
---------------------
Player to move: Player 2

Game is over!

---------------------
| A | * | B |   | D |
---------------------
| 1 |   | 2 |   |   |
---------------------
|   | E |   |   |   |
---------------------
|   |   |   | C | 3 |
---------------------
|   | 4 |   | 5 |   |
---------------------
Player to move: Player 2


(-1, 1)

Now, let's try to have our minimax agent play against a random agent starting from the beginning of the game. When you run the code cell below, you will find that our minimax agent is unable to process the state space search! In fact, __your Python kernel will crash__, because it exceeds the maximum recursion depth. The takeaway from this is that, unfortunately, a pure minimax approach is intractable for our __Neutron__ domain.

In [8]:
game = NeutronGame()
p1   = MinimaxAgent()
p2   = RandomAgent()
game.play_game(p1, p2, False)

RecursionError: maximum recursion depth exceeded while calling a Python object

### Part 2: Alpha-Beta Pruning

<h3 style="text-align: right; margin-top: -1em;">20 points</h3>

---

As we saw from part 1, while the minimax algorithm is a good start, this approach fails in our domain, due to the exponential number of states that it needs to explore. Recall from lecture that it is possible for us to improve the algorithm through the use of a pruning technique, called __alpha-beta pruning__. In particular, we use $\alpha$ to refer to the value of the best choice (i.e. highest-value) so far for MAX, while $\beta$ refers to the value of the best choice (i.e. lowest-value) so far for MIN. Alpha-beta search updates the values for $\alpha$ and $\beta$ as it goes along and prunes branches once it determines that the value of the state is worse than $\alpha$ (for MAX) or $\beta$ (for MIN).

Your second task in this assignment is to implement the above behavior. Similar to how minimax is split into 2 functions (depending on the current player), we also split the alpha-beta pruning into 2 functions. For easy reference, modified pseudocode from lecture has been reproduced below. Use this to guide your implementation. Note, we are representing $\alpha$ as the variable `a` below, and similarly, representing $\beta$ as the variable `b`.

```
alphabeta_minimax(game, state, cutoff, depth, eval_fn, a, b)
    if cutoff(state, depth) then
        return eval_fn(state, player 1), empty_action
    else
        val = -∞
        best_act = empty_action
        foreach act in actions(state)
            val' = alphabeta_maximin(game, result(state, act), cutoff, depth + 1, eval_fn, a, b)
            if val' > val then
                val = val'
                best_act = act
            if val >= b then
                return val, best_act
            a = max(a, val)
    return val, best_act
```

---

```
alphabeta_maximin(game, state, cutoff, depth, eval_fn, a, b)
    if cutoff(state, depth) then
        return eval_fn(state, player 2), empty_action
    else
        val = ∞
        best_act = empty_action
        foreach act in actions(state)
            val' = alphabeta_minimax(game, result(state, act), cutoff, depth + 1, eval_fn, a, b)
            if val' < val then
                val = val'
                best_act = act
            if val <= a then
                return val, best_act
            b = min(b, val)
    return val, best_act
```

In [9]:
"""
:Minimax search algorithm implentation using alpha-beta pruning
:type game: object (an object generated by NeutronGame class)
:type state: list (possible sates)
:type cutoff: function (optional, customized cutoff function, default by game.terminal_test)
:type depth: int (optional, recursion depth, default by 0)
:type eval_fn: function (optional, default by game.utility)
:type a: float (optional, the value of the best choice so far for MAX, default by negative infinity -> alpha)
:type b: float (optional, the value of the best choice so far for MIN, default by positive infinity -> beta)
:rtype: tuple (returns a value and an action)
"""
def alphabeta_minimax(game, state, cutoff = None, depth = 0, eval_fn = None, a = -math.inf, b = math.inf):
    # default cutoff is just a test if the state is terminal
    if cutoff == None:
        cutoff = lambda s, d : game.terminal_test(s)
    
    # default eval is just the utility value
    if eval_fn == None:
        eval_fn = game.utility
    
    # replace the line below with your code
    if cutoff(state, depth):
        return eval_fn(state, p1), []
    else:
        val = float('-inf')
        best_act = []
        for act in game.actions(state):
            val_current = alphabeta_maximin(game, game.result(state, act), cutoff, depth+1, eval_fn, a, b)[0]
            if val_current > val:
                val = val_current
                best_act = act
            if val >= b:
                return val, best_act
            a = max(a, val)
    return val, best_act

"""
:Maxmin search algorithm implentation using alpha-beta pruning
:type game: object (an object generated by NeutronGame class)
:type state: list (possible sates)
:type cutoff: function (optional, customized cutoff function, default by game.terminal_test)
:type depth: int (optional, recursion depth, default by 0)
:type eval_fn: function (optional, default by game.utility)
:type a: float (optional, the value of the best choice so far for MAX, default by negative infinity -> alpha)
:type b: float (optional, the value of the best choice so far for MIN, default by positive infinity -> beta)
:rtype: tuple (returns a value and an action)
"""
def alphabeta_maximin(game, state, cutoff = None, depth = 0, eval_fn = None, a = -math.inf, b = math.inf):
    # default cutoff is just a test if the state is terminal
    if cutoff == None:
        cutoff = lambda s, d : game.terminal_test(s)
    
    # default eval is just the utility value
    if eval_fn == None:
        eval_fn = game.utility
    
    # replace the line below with your code
    if cutoff(state, depth):
        return eval_fn(state, p2), []
    else:
        val = float('inf')
        best_act = []
        for act in game.actions(state):
            val_current = alphabeta_minimax(game, game.result(state, act), cutoff, depth+1, eval_fn, a, b)[0]
            if val_current < val:
                val = val_current
                best_act = act
            if val < a:
                return val, best_act
            b = min(b, val)
    return val, best_act

Once we have a working implementation of __alphabeta_minimax__ and __alphabeta_maximin__, we can create agents based around these algorithms. Just like the minimax agents, these agents can take the optional parameters, but let's not worry about them just yet.

In [10]:
"""
An agent that utilizes the alpha-beta minimax search algorithm. Can take two optional parameters.
cutoff, that can be used to determine when to stop the search
eval_fn, which evaluates a state's value from the perspective of a given player
"""
class AlphabetaMinimaxAgent:
    def __init__(self, cutoff = None, eval_fn = None):
        self.cutoff  = cutoff
        self.eval_fn = eval_fn

    def play_turn(self, game, state):
        _, best_act = alphabeta_minimax(game, state, cutoff = self.cutoff, eval_fn = self.eval_fn)
        return best_act

"""
An agent that utilizes the alpha-beta maximin search algorithm. Can take two optional parameters.
cutoff, that can be used to determine when to stop the search
eval_fn, which evaluates a state's value from the perspective of a given player
"""
class AlphabetaMaximinAgent:
    def __init__(self, cutoff = None, eval_fn = None):
        self.cutoff  = cutoff
        self.eval_fn = eval_fn

    def play_turn(self, game, state):
        _, best_act = alphabeta_maximin(game, state, cutoff = self.cutoff, eval_fn = self.eval_fn)
        return best_act

As a test, we are going to start from a particular state partway through a game. This particular state was written so that the alpha-beta agent will be able to playout until the end. Run the code cell below to check if your implementation is able to win.

In [11]:
custom_board = {
    (1,1) : PLAYER_1[0], (1,2) : BLANK, (1,3) : PLAYER_1[1], (1,4) : PLAYER_1[2], (1,5) : PLAYER_1[3],
    (2,1) : PLAYER_2[0], (2,2) : NEUTRON, (2,3) : PLAYER_2[1], (2,4) : BLANK, (2,5) : BLANK,
    (3,1) : BLANK, (3,2) : PLAYER_1[4], (3,3) : BLANK, (3,4) : BLANK, (3,5) : BLANK,
    (4,1) : BLANK, (4,2) : BLANK, (4,3) : BLANK, (4,4) : BLANK, (4,5) : PLAYER_2[2],
    (5,1) : BLANK, (5,2) : PLAYER_2[3], (5,3) : BLANK, (5,4) : PLAYER_2[4], (5,5) : BLANK,
}
custom_state = GameState(board = custom_board, player = PLAYER_1, is_initial = False, utility_value = 0)
game         = NeutronGame(custom_state)
p1           = AlphabetaMinimaxAgent()
p2           = RandomAgent()
game.play_game(p1, p2, True)


---------------------
| A |   | B | C | D |
---------------------
| 1 | * | 2 |   |   |
---------------------
|   | E |   |   |   |
---------------------
|   |   |   |   | 3 |
---------------------
|   | 4 |   | 5 |   |
---------------------
Player to move: Player 1

---------------------
| A | * | B |   | D |
---------------------
| 1 |   | 2 |   |   |
---------------------
|   | E |   |   |   |
---------------------
|   |   |   | C | 3 |
---------------------
|   | 4 |   | 5 |   |
---------------------
Player to move: Player 2

Game is over!

---------------------
| A | * | B |   | D |
---------------------
| 1 |   | 2 |   |   |
---------------------
|   | E |   |   |   |
---------------------
|   |   |   | C | 3 |
---------------------
|   | 4 |   | 5 |   |
---------------------
Player to move: Player 2


(-1, 1)

Now, let's try to have our alpha-beta agent play against a random agent starting from the beginning of the game. When you run the code cell below, you will find that our alpha-beta agent is also unable to process the state space search, much like minimax. Once again, __your Python kernel will crash__. The takeaway from this is that even alpha-beta pruning does not help to make our __Neutron__ domain tractable for adversarial search.

In [12]:
game = NeutronGame()
p1   = AlphabetaMinimaxAgent()
p2   = RandomAgent()
game.play_game(p1, p2, False)

RecursionError: maximum recursion depth exceeded while calling a Python object

### Part 3: Heuristics

<h3 style="text-align: right; margin-top: -1em;">40 points</h3>

---

Is there any way to salvage our adversarial search? It turns out, there is! We can cut off our search early and apply a heuristic function to evaluate states, effectively treating non-terminal states as if they were terminal. Of course, this begs the question, what makes a good cutoff test? And what makes a good evaluation function?

This is your third task for this assignment. Come up with your own heuristics for the cutoff and evaluation functions so that our agents can finally play the game from start to finish. `custom_cutoff` should return a Boolean value and `custom_eval` should return an integer (positive if the state will lead to a win, negative if the state will lead to a loss).

For reference, here is some documentation on a game state:
> A state for this game is a 4-tuple, (`board`, `player`, `is_initial`, `utility_value`).
> - `board`, a dictionary with (i,j) coordinates as the key, and the piece as
  the value. Valid coordinate values for i and j are 1 to 5, inclusive. The piece
  should refer to one of the variables `PLAYER_1[n]`, `PLAYER_2[n]`, `NEUTRON`, or
  `BLANK`, where `n` can have a value from 0 to 4, inclusive.
> - `player`, refers to the player that needs to move in this state. Should reference
  one of the variables `PLAYER_1` or `PLAYER_2`.
> - `is_initial`, a Boolean value that determines whether or not this state is the
  initial state of the game.
> - `utility_value`, an integer value that represents the utility value for the
  state.

In [52]:
"""
:To avoid TLE, we customized the following cutoff function:
:The game will be terminated if, 1. pass the terminal test (i.e., someone wins) OR, 2. recursion depth has reached to 2
:type state: list (possible states)
:type depth: int (recursion depth)
:rtype: boolean
"""
def custom_cutoff(state, depth):
    # replace the line below with your code
    return game.terminal_test(state) or depth >= 2

"""
:My heuristic function is designed to make sure that in the current state,
:1. the more number of actions that current player could take, the better
:2. the more number of the current player's pieces prevent the neutron from the opponent's home line, the better
:3. the less number of the opponent's pieces prevent the neutron from the player's home line, the better
:No.2 and No.3 conditions are considered weigh more than No.1, so we multiply them by some parameters (5, 10)
:type state: list (possible states)
:type player: list (player 1 or player 2)
:rtype: float
"""
def custom_eval(state, player):
    # replace the line below with your code
    for loc, blank_piece in state.board.items():
        if blank_piece == NEUTRON:
            ### neutron coordinates in current state
            neutron_x, neutron_y = loc
            break
            
    block_neutron_pieces_num_1 = block_neutron_pieces_num_2 = 0
    for x in range(1, neutron_x):
        if state.board[(x, neutron_y)] in PLAYER_2:
            ### number of player 2's pieces that "blocks" player 1 from putting the neutron to the home line
            block_neutron_pieces_num_2 += 1
    for x in range(neutron_x, 6):
        if state.board[(x, neutron_y)] in PLAYER_1:
            ### number of player 1's pieces that "blocks" player 2 from putting the neutron to the home line
            block_neutron_pieces_num_1 += 1
    
    heuristic = len(game.actions(state)) + 5 * block_neutron_pieces_num_1 - 10 * block_neutron_pieces_num_2
    
    return heuristic

Now let's evaluate your heuristics! First, we will try the minimax agent against the random agent. (Your kernel should not crash this time.)

In [53]:
game  = NeutronGame()
p1    = MinimaxAgent(custom_cutoff, custom_eval)
p2    = RandomAgent()
start = time.process_time()
game.play_game(p1, p2, False)
end = time.process_time()
print('\nTime elapsed:', end - start)


Game is over!

---------------------
| B |   |   | D | * |
---------------------
|   | 2 | C |   |   |
---------------------
|   |   |   |   |   |
---------------------
| A | 3 |   |   | E |
---------------------
| 1 |   |   | 4 | 5 |
---------------------
Player to move: Player 1

Time elapsed: 0.5625


Second, we will try the alpha-beta agent against the random agent. (Again, your kernel should not crash.)

In [54]:
game  = NeutronGame()
p1    = AlphabetaMinimaxAgent(custom_cutoff, custom_eval)
p2    = RandomAgent()
start = time.process_time()
game.play_game(p1, p2, False)
end = time.process_time()
print('\nTime elapsed:', end - start)


Game is over!

---------------------
| A |   |   |   | E |
---------------------
|   | B | C |   |   |
---------------------
| 3 |   |   |   |   |
---------------------
|   |   |   | D |   |
---------------------
| 1 | * | 2 | 4 | 5 |
---------------------
Player to move: Player 1

Time elapsed: 0.15625


### Part 4: Experiments & Questions

<h3 style="text-align: right; margin-top: -1em;">15 points</h3>

---

At the moment, the game has been using a fixed move ordering of up, down, left, then right. However, it is possible that this move ordering affects the overall performance of our adversarial search. First, let's see how this move ordering affects the number of states generated. To do this, we will play the game between the alpha-beta (with heuristics) agent and the random agent. We will do this repeatedly and then generate a link to a graph of the data. Now, what if we change the move ordering? In the code cell below, provide your own ordering by changing the value of `custom_move_order`, then run the cell. __How did your ordering affect the number of states generated compared to the initial fixed ordering?__ Write your response in the raw text cell below the code cell.

_Note_: Running the cell below will take some time.

For reference, here is the list of moves as notated in the game logic and what they mean:
+ (-1,0) is UP
+ (1,0) is DOWN
+ (0,-1) is LEFT
+ (0,1) is RIGHT

Please ensure your choice for `custom_move_order` is different from the initial fixed move order (up, down, left, right).

In [56]:
# provide your custom move order below
# this should be a list of 4 tuples
custom_move_order = [(0,-1), (1,0), (0,1), (-1,0)]

# run trials for number of states generated using the initial move order
num_trials = 20
results    = []
for m in range(num_trials):
    avg = 0.0
    for n in range(num_trials):
        game = NeutronGame()
        p1   = AlphabetaMinimaxAgent(custom_cutoff, custom_eval)
        p2   = RandomAgent()
        game.play_game(p1, p2, False, True)
        avg += game.num_states_gen
    results.append(avg / num_trials)
print('This link is for the number of states generated using the initial move order:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

# run trials for number of states generated using the custom move order
results = []
for m in range(num_trials):
    avg = 0.0
    for n in range(num_trials):
        game = NeutronGame(move_order = custom_move_order)
        p1   = AlphabetaMinimaxAgent(custom_cutoff, custom_eval)
        p2   = RandomAgent()
        game.play_game(p1, p2, False, True)
        avg += game.num_states_gen
    results.append(avg / num_trials)
print('\nThis link is for the number of states generated using your custom move order:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

This link is for the number of states generated using the initial move order:
https://www.wolframalpha.com/input/?i=plot+%7B4659.1,3585.4,4637.1,5143.2,5583.45,4499.75,5561.5,3560.25,3396.95,5403.75,3865.2,4975.0,4842.6,6335.45,4428.0,3555.4,3840.75,3930.3,4156.5,4884.3%7D

This link is for the number of states generated using your custom move order:
https://www.wolframalpha.com/input/?i=plot+%7B3848.0,6646.5,3554.05,4703.7,4872.1,4741.5,5145.4,5342.4,4131.25,4432.45,5655.45,4362.3,5518.75,5414.05,5998.05,4702.1,6083.95,4507.2,6027.0,5183.85%7D


**Answer:**<br/>
From the generated plots we can see that after we changed the order somehow, the number of states is **increased** (on average about 4500 using initial order and 5000 after we changed the order)

For a second experiment, let us take a look at the ordering of our agents. Thus far, we have always been placing our agents as the first player of the game. __What if we place our agent as the second player instead? Does it affect the win rate? How about the number of states generated during the search?__ Run the code cell below to run the experiment and generate links to plot the data. Analyze these plots and provide a short answer in the raw text cell below.

_Note_: Running the cell below will take some time.

In [55]:
# run trials for win rate as first player
num_trials = 20
results    = []
for m in range(num_trials):
    win = 0.0
    for n in range(num_trials):
        game = NeutronGame()
        p1   = AlphabetaMinimaxAgent(custom_cutoff, custom_eval)
        p2   = RandomAgent()
        v    = game.play_game(p1, p2, False, True)
        if v == (1,2) or v == (-1,1):
            win += 1.0
    results.append(win / num_trials)
print('This link is for the win rate as first player:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

# run trials for win rate as second player
results = []
for m in range(num_trials):
    win = 0.0
    for n in range(num_trials):
        game = NeutronGame()
        p1   = RandomAgent()
        p2   = AlphabetaMaximinAgent(custom_cutoff, custom_eval)
        v    = game.play_game(p1, p2, False, True)
        if v == (1,1) or v == (-1,2):
            win += 1.0
    results.append(win / num_trials)
print('\nThis link is for the win rate as second player:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

# run trials for number of states generated as first player
results = []
for m in range(num_trials):
    avg = 0.0
    for n in range(num_trials):
        game = NeutronGame()
        p1   = AlphabetaMinimaxAgent(custom_cutoff, custom_eval)
        p2   = RandomAgent()
        game.play_game(p1, p2, False, True)
        avg += game.num_states_gen
    results.append(avg / num_trials)
print('\nThis link is for the number of states generated as first player:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

# run trials for number of states generated as second player
results = []
for m in range(num_trials):
    avg = 0.0
    for n in range(num_trials):
        game = NeutronGame()
        p1   = RandomAgent()
        p2   = AlphabetaMaximinAgent(custom_cutoff, custom_eval)
        game.play_game(p1, p2, False, True)
        avg += game.num_states_gen
    results.append(avg / num_trials)
print('\nThis link is for the number of states generated as second player:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

This link is for the win rate as first player:
https://www.wolframalpha.com/input/?i=plot+%7B0.7,0.9,0.7,0.8,0.85,0.8,0.8,0.8,0.8,0.8,0.65,0.65,0.85,0.85,0.75,0.8,0.6,0.85,0.95,0.7%7D

This link is for the win rate as second player:
https://www.wolframalpha.com/input/?i=plot+%7B0.7,0.75,0.75,0.7,0.6,0.65,0.65,0.5,0.55,0.55,0.6,0.45,0.5,0.7,0.65,0.6,0.65,0.4,0.7,0.65%7D

This link is for the number of states generated as first player:
https://www.wolframalpha.com/input/?i=plot+%7B5023.65,4753.8,5267.55,4469.2,5253.0,4187.0,4926.35,4727.55,3796.85,4286.2,5006.3,4778.05,3575.95,4070.3,5524.75,4294.1,5254.35,4749.3,4103.65,4694.1%7D

This link is for the number of states generated as second player:
https://www.wolframalpha.com/input/?i=plot+%7B1255.65,1160.1,1315.7,1197.0,1433.45,1069.65,1317.9,1476.05,1395.8,1157.4,1218.4,1300.95,1160.4,1161.9,1059.15,1601.25,1651.9,1105.25,1648.3,1335.4%7D


**Answer:**<br/>
From the generated plot we can see that if we change the agent order, **the winning rate decreased from 0.8 to 0.6 on average; the number of states decreased from 4600 to 1300 on average.**  

For one final experiment, let's look at __how the depth limit as the cutoff test can affect our search__. Run the code cell below, analyze the data, and provide your answer in the raw text cell.

_Note_: Running the cell below will take some time.

In [57]:
# run trials for win rate
max_depth  = 5
results    = []
num_trials = 30
for depth in range(max_depth):
    exp_cutoff = lambda s, d : d >= depth + 1
    win        = 0.0
    for n in range(num_trials):
        game = NeutronGame()
        p1   = AlphabetaMinimaxAgent(exp_cutoff, custom_eval)
        p2   = RandomAgent()
        v    = game.play_game(p1, p2, False, True)
        if v == (1,2) or v == (-1,1):
            win += 1.0
    results.append(win / num_trials)
print('This link is for the average win rate as the depth limit increases:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

# run trials for number of states generated
results = []
for depth in range(max_depth):
    exp_cutoff = lambda s, d : d >= depth + 1
    avg        = 0.0
    for n in range(num_trials):
        game = NeutronGame()
        p1   = AlphabetaMinimaxAgent(exp_cutoff, custom_eval)
        p2   = RandomAgent()
        game.play_game(p1, p2, False, True)
        avg += game.num_states_gen
    results.append(avg / num_trials)
print('\nThis link is for the average number of states generated as the depth limit increases:')
s = 'https://www.wolframalpha.com/input/?i=plot+%7B'
for r in results:
    s += str(r) + ','
print(s[:-1] + '%7D')

This link is for the average win rate as the depth limit increases:
https://www.wolframalpha.com/input/?i=plot+%7B0.5,0.9,0.7,0.9,0.9333333333333333%7D

This link is for the average number of states generated as the depth limit increases:
https://www.wolframalpha.com/input/?i=plot+%7B297.73333333333335,1432.9333333333334,12053.4,53133.333333333336,314442.9666666667%7D


**Answer:**<br/>
From the plots we can find that generally, **increasing depth limit can definitely result in an increasing of winning rate. However, the winning rate converges when the depth limit reaches to some point** (2-3 in this case, that's why I set the customized cutoff depth to be 2).<br/>
**For average number of states, it keeps increasing as depth goes up.** Also, it seems that the speed is growing exponentially somehow (won't converge).

### Part 5: Just for Fun
---

Finally, to close this assignment, you are now tasked with creating 2 additional game states. For the first game state, our pure minimax agent (meaning no heuristics) should be able to playout as the first player. For the second game state, the pure alpha-beta agent should be able to playout as the first player, but the pure minimax agent should not be able to. Good luck! :)

__Do not use the game state from earlier in the assignment!__

In [65]:
minimax_board = {
    (1,1) : PLAYER_1[0], (1,2) : PLAYER_1[1], (1,3) : PLAYER_1[2], (1,4) : PLAYER_1[3], (1,5) : PLAYER_1[4],
    (2,1) : BLANK, (2,2) : BLANK, (2,3) : BLANK, (2,4) : BLANK, (2,5) : BLANK,
    (3,1) : BLANK, (3,2) : BLANK, (3,3) : NEUTRON, (3,4) : BLANK, (3,5) : BLANK,
    (4,1) : BLANK, (4,2) : BLANK, (4,3) : BLANK, (4,4) : BLANK, (4,5) : BLANK,
    (5,1) : PLAYER_2[0], (5,2) : PLAYER_2[1], (5,3) : PLAYER_2[2], (5,4) : PLAYER_2[3], (5,5) : PLAYER_2[4]
}
minimax_state = GameState(board = minimax_board, player = PLAYER_1, is_initial = False, utility_value = 0)
game          = NeutronGame(minimax_state)
p1            = MinimaxAgent(custom_cutoff, custom_eval)
p2            = RandomAgent()
v             = game.play_game(p1, p2, False)
if v == (1,2) or v == (-1,1):
    print('Congrats! The minimax agent was able to win!')

alphabeta_board = {
    (1,1) : PLAYER_1[0], (1,2) : PLAYER_1[1], (1,3) : PLAYER_1[2], (1,4) : PLAYER_1[3], (1,5) : PLAYER_1[4],
    (2,1) : BLANK, (2,2) : BLANK, (2,3) : BLANK, (2,4) : BLANK, (2,5) : BLANK,
    (3,1) : BLANK, (3,2) : BLANK, (3,3) : NEUTRON, (3,4) : BLANK, (3,5) : BLANK,
    (4,1) : BLANK, (4,2) : BLANK, (4,3) : BLANK, (4,4) : BLANK, (4,5) : BLANK,
    (5,1) : PLAYER_2[0], (5,2) : PLAYER_2[1], (5,3) : PLAYER_2[2], (5,4) : PLAYER_2[3], (5,5) : PLAYER_2[4]
}
alphabeta_state = GameState(board = alphabeta_board, player = PLAYER_1, is_initial = False, utility_value = 0)
game            = NeutronGame(alphabeta_state)
p1              = AlphabetaMinimaxAgent(custom_cutoff, custom_eval)
p2              = RandomAgent()
v               = game.play_game(p1, p2, False)
if v == (1,2) or v == (-1,1):
    print('Congrats! The alpha-beta agent was able to win!')
    print('But you make sure that your minimax agent cannot [as in, it should kill the Python kernel].')


Game is over!

---------------------
|   | 2 | D | * |   |
---------------------
| B |   | 3 |   | C |
---------------------
| E |   |   |   |   |
---------------------
| 1 |   |   |   | 4 |
---------------------
|   |   |   | A | 5 |
---------------------
Player to move: Player 2
Congrats! The minimax agent was able to win!

Game is over!

---------------------
|   | B | D |   | * |
---------------------
|   |   |   | 4 |   |
---------------------
|   |   | C |   | E |
---------------------
|   |   | A |   |   |
---------------------
| 1 | 2 |   | 3 | 5 |
---------------------
Player to move: Player 2
Congrats! The alpha-beta agent was able to win!
But you make sure that your minimax agent cannot [as in, it should kill the Python kernel].


### Submission
---

You will only be submitting your Jupyter notebook file, *hw3.ipynb*. Do not worry about submitting the additional *game_tree.png* file. Furthermore, as a reminder, part of your grade is your documentation. Each of the functions you implemented as part of this assignment **must** be documented.

Please upload your *hw3.ipynb* file to CMS by **Monday, March 4 @ 1:24pm**.